In [1]:
import sys
#print(sys.version)

In [2]:
sg.theme_list()

['Black',
 'BlueMono',
 'BluePurple',
 'BrightColors',
 'BrownBlue',
 'Dark',
 'Dark2',
 'DarkAmber',
 'DarkBlack',
 'DarkBlack1',
 'DarkBlue',
 'DarkBlue1',
 'DarkBlue10',
 'DarkBlue11',
 'DarkBlue12',
 'DarkBlue13',
 'DarkBlue14',
 'DarkBlue15',
 'DarkBlue16',
 'DarkBlue17',
 'DarkBlue2',
 'DarkBlue3',
 'DarkBlue4',
 'DarkBlue5',
 'DarkBlue6',
 'DarkBlue7',
 'DarkBlue8',
 'DarkBlue9',
 'DarkBrown',
 'DarkBrown1',
 'DarkBrown2',
 'DarkBrown3',
 'DarkBrown4',
 'DarkBrown5',
 'DarkBrown6',
 'DarkBrown7',
 'DarkGreen',
 'DarkGreen1',
 'DarkGreen2',
 'DarkGreen3',
 'DarkGreen4',
 'DarkGreen5',
 'DarkGreen6',
 'DarkGreen7',
 'DarkGrey',
 'DarkGrey1',
 'DarkGrey10',
 'DarkGrey11',
 'DarkGrey12',
 'DarkGrey13',
 'DarkGrey14',
 'DarkGrey15',
 'DarkGrey2',
 'DarkGrey3',
 'DarkGrey4',
 'DarkGrey5',
 'DarkGrey6',
 'DarkGrey7',
 'DarkGrey8',
 'DarkGrey9',
 'DarkPurple',
 'DarkPurple1',
 'DarkPurple2',
 'DarkPurple3',
 'DarkPurple4',
 'DarkPurple5',
 'DarkPurple6',
 'DarkPurple7',
 'DarkRed',
 'Da

In [10]:
import PySimpleGUI as sg
import os
import subprocess
import threading 
from pymol import cmd
import pymol
import shutil
import time
import traceback # 에러 처리용
sg.theme("SystemDefault")  # No gray windows please!
# 컴포넌트 정의
# STEP 1 define the layout


left_column = [
    [sg.Image(r"sample_img.png", size=(200, 200))],
    [sg.Text("리셉터 파일을 업로드", size=(30, 1), justification="center")],
    [sg.Input("파일 경로 입력...", size=(25, 1)), sg.FileBrowse(button_text="찾아보기...",key='file_directory_btn')],
    [sg.Button("업로드하기", key = 'file_upload_btn')],
    [sg.Text("업로드된 리셉터 파일 이름 : "), sg.Text("----", key = 'receptor_name')],
    [sg.Text("_" * 35)],
    [sg.Text("추천되는 파라미터", size=(30, 1), justification="center")],
    [
        sg.Text(
            "exhaustiveness : 8 \n Grid 8 8 8 ", size=(30, 3), justification="center", key='-PARAMETER_VALUE-'
        )
    ],
    #[sg.Table()]
]


right_column = [
    [sg.Text("col Row 1")],
    [
        sg.Frame(
            "My Frame Title",
            [
                [sg.T("리간드 파일 선택")],
                [
                    sg.Column(
                        [
                            [sg.Image(r"folder_icon.png", size=(60, 60))],
                            [sg.Text("리셉터 폴더 선택")],
                        ],
                        element_justification="center",
                    ),
                    sg.Column(
                        [
                            [sg.Image(r"db_icon.png", size=(60, 60))],
                            [sg.Text("온라인 DB 사용")],
                        ],
                        element_justification="center",
                    ),
                ],
                [
                    sg.Input("폴더 경로 입력...", size=(25, 1)),
                    sg.FolderBrowse(button_text="찾아보기...", key="-FOLDER BROWSE-"),
                    sg.Button("업로드하기", key="-UPLOAD-")
                ],
            ],
            element_justification="center",
        )
    ],
    [
        sg.Table(
            [["Dapagliflozin (1)", "대기중", "--:--:--", "--", "--"], [4, 5, 6, 7, 8]],
            ["리간드 이름", "Docking 수행", "Docking 수행 시간", "결과 평가", "사용한 파라미터"],
            num_rows=3,
            justification="center",
            key="-TABLE-",
        )
    ],
    [sg.Text("What you print will display below:")],
    [sg.Output(size=(50, 10), key="-OUTPUT-")],
    [sg.In(key="-IN-")],
    [sg.Button("Run Vina"), sg.Button('Eval RMSD'), sg.Button("Clear")],
]


layout = [
    [
        sg.Column(left_column, expand_y=True),
        sg.VerticalSeparator(pad=None),
        sg.Column(right_column, expand_y=True),
    ],
    [sg.Button("debug Button"), \
     sg.Button("Test Case 1(vina 설치 테스트)", key = 'test_case_1'),\
     sg.Button("Test Case 2(vina 오동작 테스트)", key = 'test_case_2'), \
     sg.Button("Test Case 3(Pymol 전처리 여부 테스트)", key = 'test_case_3'), \
     sg.Button("Test Case 4(Pymol 전처리 수행 테스트)", key = 'test_case_4'), \
     sg.Button("Exit")],
]

column = [[layout]]

# STEP 2 - create the window
window = sg.Window("My new window", column, grab_anywhere=True)

table = window["-TABLE-"]
folder_input = window["-FOLDER BROWSE-"]

def subprocess_vina_test_install_check(): ## 사용자 시스템에 Vina가 설치되어있는지 테스트하는 함수입니다.
    try:
        command = "vina --help"
        folder = ".\\SGLT_sample\\"
        p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
        return_code = p.wait() ## 프로세스 리턴코드 입니다. 프로세스가 잘 동작했는지 확인합니다.
        if return_code !=0 : ## 만약 리턴코드가 0이 아니면 (비정상 동작)
            raise subprocess.CalledProcessError(return_code, command)
        print(f'[테스트 성공] : Vina가 시스템에 설치되어 있습니다.')
        
        # CMD의 내용을 확인할 필요가 있을경우 하위 주석을 해제하여 사용합니다.
        #for line in p.stdout:
        #    print(line.decode('cp949').strip())       
    except subprocess.CalledProcessError:
        print(f'[테스트 실패] : 프로그램에 Vina가 설치되지 않은 것 같습니다. 에러 코드 :{return_code}')
        sg.popup(f'[테스트 실패] : 프로그램에 Vina가 설치되지 않은 것 같습니다. 에러 코드 :{return_code}', title='테스트 실패')
        
def subprocess_vina_test_malfunction(): ## 사용자 시스템에 Vina가 오동작하는것을 테스트하는 함수입니다. 
    ## 30초를 기준으로 넘으면 타임아웃으로 간주합니다. 
    try:
        command = "vina --config config2.txt --log log.txt --out output.pdbqt"
        folder = ".\\SGLT_sample\\"
        timeout_secs = 30 ## !!타임아웃 몇초로 할 것인지 결정 ##
        p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
        return_code = p.wait(timeout=timeout_secs) ## 프로세스 리턴코드 입니다. 프로세스가 잘 동작했는지 확인합니다.
        if return_code !=0 : ## 만약 리턴코드가 0이 아니면 (비정상 동작)
            raise subprocess.CalledProcessError(return_code, command)
        for line in p.stdout:
            print(line.decode('cp949').strip())
    except subprocess.TimeoutExpired:
        print(f'[테스트 실패] : Vina가 응답하지 않는 것 같습니다... 타임아웃 : {timeout_secs}초')
        sg.popup(f'[테스트 실패] : Vina가 응답하지 않는 것 같습니다... 타임아웃 : {timeout_secs}초', title='테스트 실패')
    except subprocess.CalledProcessError:
        print(f'[테스트 실패] : 프로그램에 Vina가 설치되지 않은 것 같습니다. 에러 코드 :{return_code}')
        sg.popup(f'[테스트 실패] : 프로그램에 Vina가 설치되지 않은 것 같습니다. 에러 코드 :{return_code}', title='테스트 실패')

def pymol_preprocess_check(): # pymol API를 통해 전처리 되었는지 여부를 확인하는 함수입니다.
    #pymol.finish_launching()
    # Load the protein file
    cmd.cd('.\SGLT_sample')
    pymol.cmd.load('7vsi.pdbqt') # 현재 하드코딩된 단백질 파일명. 추후 수정 필요

    # Check for the presence of hydrogens and solvent molecules
    hydrogens = pymol.cmd.count_atoms('name H*')
    solvent = pymol.cmd.count_atoms('solvent')

    if hydrogens > 0 or solvent > 0:
        print(f'이 단백질은 이미 전처리되었습니다. \n수소 원자 수 : {hydrogens}, 솔벤트 원자 수 : {solvent}')
    else:
        print('이 단백질은 아직 전처리되지 않았습니다.')
    cmd.cd('../')
    
def pymol_preprocess(): # 단백질을 전처리하는 함수입니다.
    # 단백질 파일 불러오기
    cmd.cd('.\SGLT_sample')
    pymol.cmd.load('7vsi.pdbqt') # 현재 하드코딩된 단백질 파일명. 추후 수정 필요
    
    # 전처리 실시
    pymol.cmd.h_add(selection='all')
    pymol.cmd.remove('(solvent)')
    pymol.cmd.sort()
    
    print('파일 전처리 성공')
    
    # 전처리된 파일 저장
    pymol.cmd.save('protein_preprocessed.pdb')
    
    # 워킹디렉토리 초기화 
    cmd.cd('../')

def upload_ligand_file(): ## 리간드 업로드 버튼
    folder_path = values["-FOLDER BROWSE-"]
    if folder_path:
        ligands = get_ligand_files(folder_path)
        copy_ligand_files(folder_path, ligands)
        table.update(values=ligands)
        return " "
    else:
        sg.popup('폴더가 선택되지 않았습니다.', title='오류')
        return "폴더 경로를 선택해주세요."
    
def get_ligand_files(folder_path):
    ligand_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith((".pdb", ".sdf", ".mol2")):
            file_name, file_type = os.path.splitext(filename)
            start_time = time.time()
            # (Docking 작업 수행 추가해야 함.)
            elapsed_time = time.time() - start_time  # Docking 작업 수행 시간 계산
            docking_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
            ligand_files.append([file_name, "대기중", docking_time, "--", "--"])
    return ligand_files

def copy_ligand_files(folder_path, ligand_files): ## 리간드 파일 복사
    target_folder = '.\ligand'
    os.makedirs(target_folder, exist_ok=True)
    for filename in os.listdir(folder_path):
        if filename.endswith((".pdb", ".sdf", ".mol2")):
            source_file = os.path.join(folder_path, filename)
            target_file = os.path.join(target_folder, filename)
            shutil.copy2(source_file, target_file) # 파일 복사

def perform_docking(): ## Vina를 호출해서 도킹을 수행하는 함수입니다, 현재 파일명 config2.txt에 하드코딩된 상태.##
    command = "vina --config config2.txt --log log.txt --out output.pdbqt"
    folder = ".\\SGLT_sample\\"
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
    for line in p.stdout:
        print(line.decode('cp949').strip())
    ## 결과 파일 스플릿 하기 ##
    command = "vina_split --input output.pdbqt"
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
    for line in p.stdout:
        print(line.decode('cp949').strip())

def eval_RMSD(): ## 파이몰 API로 RMSD를 계산하는 함수입니다. 현재 파일명 여기에 하드코딩. ## 
    cmd.cd('.\SGLT_sample')
    cmd.load('output_ligand_5.pdbqt')
    cmd.load('output_ligand_6.pdbqt')
    rms = cmd.pair_fit('output_ligand_5', 'output_ligand_6')
    print(rms)
    cmd.cd('../')

def upload_file(): ## 리셉터 업로드 버튼 
    src_file = values['file_directory_btn'] # 
    dst_dir = './uploaded_file/'
    if(src_file != ''): # 파일 선택여부 검사
        fileName = os.path.basename(src_file) # 파일명
        fileType = os.path.splitext(src_file)[1] # 파일타입
        if(fileType == '.pdbqt'):
            shutil.copy(src_file, dst_dir) # 파일 복사
            window['receptor_name'].update(fileName)
            updated_params = "exhaustiveness : 10 \n Grid 10 10 10 "
            window['-PARAMETER_VALUE-'].update(updated_params)
            return "파일을 업로드하였습니다. : " + fileName + fileType
        else:
            sg.popup('파일 확장자가 pdbqt가 아닙니다.', title = '오류')
            return "파일 확장자가 pdbqt가 아닙니다."
    else:
        sg.popup('파일이 선택되지 않았습니다.', title='오류')
        return "파일 경로를 선택해주세요."
# STEP3 - the event loop

## 주요 버튼에 이벤트 할당하는 곳 ## 
while True:
    ( event, values ) = window.read()  # Read the event that happened and the values dictionary
    #print(event, values)
    try:
        
        if event in ( None, "Exit",):  # If user closeddow with X or if user clicked "Exit" button then exit
            break
        if event == "debug Button":
            #f = os.popen("dir")
            #print("You pressed the button" + f.read())
            print(values['file_directory_btn'])
        if event == "test_case_1":
            subprocess_vina_test_install_check()
        if event == "test_case_2":
            subprocess_vina_test_malfunction()
        if event == "test_case_3":
            pymol_preprocess_check()
        if event == "test_case_4":
            pymol_preprocess()
        if event == "-UPLOAD-": ## 리간드 업로드 버튼
            print(upload_ligand_file())
        if event == "file_upload_btn": ## 리셉터 업로드 버튼 
            print(upload_file())
        if event == 'Run Vina':
            t = threading.Thread(target=perform_docking)
            t.start()
        if event == 'Eval RMSD':
            t = threading.Thread(target=eval_RMSD)
            t.start()
        if event == 'Clear':
            window['-OUTPUT-'].update('')
        pass
    except Exception as e :
        sg.popup_error(f'An error occured: {e}')
        #print(f'Type: {type(e)}')
        #print(f'Message: {str(e)}')
        tb = e.__traceback__
        line_number = tb.tb_lineno
        print(f'Error occurred on line {line_number}: {e}')
        #print(f'Error occurred: {e.args}')
        traceback.print_exc()
window.close()


In [2]:
from pymol import cmd

cmd.load('output_ligand_1.pdbqt')
cmd.load('output_ligand_2.pdbqt')

rms = cmd.pair_fit('output_ligand_1', 'output_ligand_2')

print(rms)

kerneltest


In [6]:
import shutil

src_file = 'C:/Users/jundo/Desktop/Git/Graduation_project/db_icon.png'
dst_dir = './db_icon2.png'

shutil.copy(src_file, dst_dir)

'./db_icon2.png'

In [7]:
import PySimpleGUI as sg

layout = [[sg.Text('Upload a file')],
          [sg.Input(key='file'), sg.FileBrowse()],
          [sg.Button('Upload'), sg.Button('Exit')],
          [sg.Listbox(values=[], size=(40, 10), key='listbox')]]

window = sg.Window('Window Title', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    elif event == 'Upload':
        filename = values['file']
        if filename:
            window['listbox'].update(values=[*values['listbox'], filename])
        else:
            sg.popup('Please choose a file')

window.close()